In [1]:
from linkingtool import RESources as RES
import pandas as pd
import geopandas as gpd

In [2]:
res_resutls=pd.read_csv('results/linking/resource_options_wind.csv')
cfp24=pd.read_csv('ROD_2024/BC_CFP24.csv')

In [3]:
res_resutls.columns

Index(['cluster_id', 'lcoe', 'capex', 'fom', 'vom', 'CF_mean', 'Cluster_No',
       'potential_capacity', 'Region', 'nearest_station',
       'nearest_station_distance_km', 'Rank', 'Operational_life'],
      dtype='object')

In [4]:
cfp24.rename(columns={'old_name': 'new_name', 'another_old_name': 'another_new_name'}, inplace=True)


In [5]:
cfp24.columns

Index(['ERA5_cell', 'project_name', 'longitude', 'latitude', 'regional_system',
       'expected_cod', 'poi', 'potential_capacity', 'annual_energy_GWh',
       'Region', 'resource_type', 'capex', 'Operational_life', 'vom', 'fom'],
      dtype='object')

In [6]:
from linkingtool.hdf5_handler import DataHandler
import linkingtool.visuals as vis
store=f"data/store/resources_BC.h5"
datahandler=DataHandler(store)

In [7]:
datahandler.show_tree(store)

Structure of HDF5 file: data/store/resources_BC.h5
[Group] boundary
[Group] cells
[Group] clusters
  └─ [Group] clusters/solar
  └─ [Group] clusters/wind
[Group] cost
  └─ [Group] cost/atb
  └─   └─ [Group] cost/atb/bess
  └─   └─ [Group] cost/atb/solar
  └─   └─ [Group] cost/atb/wind
[Group] dissolved_indices
  └─ [Group] dissolved_indices/solar
  └─ [Group] dissolved_indices/wind
[Group] substations
[Group] timeseries
  └─ [Group] timeseries/clusters
  └─   └─ [Group] timeseries/clusters/solar
  └─   └─ [Group] timeseries/clusters/wind
  └─ [Group] timeseries/solar
  └─ [Group] timeseries/wind
[Group] units


In [8]:
cells=datahandler.from_store('cells')
tss=datahandler.from_store('timeseries/solar')
tsw=datahandler.from_store('timeseries/wind')

In [9]:
CFP24=gpd.read_file('ROD_2024/BC_CFP24.geojson',Index='ERA5_cell')
CFP24.index=CFP24['ERA5_cell']

In [10]:
CFP24[CFP24['resource_type']=='solar']

,project_name,ERA5_cell,longitude,latitude,regional_system,expected_cod,poi,potential_capacity,annual_energy_GWh,Region,resource_type,capex,Operational_life,vom,fom,geometry
ERA5_cell,,,,,,,,,,,,,,,,
Thompson-Nicola_-120.75_50.5,ShTSaQU Solar Project,Thompson-Nicola_-120.75_50.5,-120.8426,50.5481,Kelly Nicola,30-Sep-30,1L203,104.0,215.0,Thompson-Nicola,solar,1482.68328,20.0,23.7656,0.0,POINT (-120.84260 50.54810)


In [11]:
tss[CFP24[CFP24['resource_type']=='solar'].index].to_csv('ROD_2024/BCH_CFP24_solar_ts.csv')

In [12]:
tsw[CFP24[CFP24['resource_type']=='wind'].index].to_csv('ROD_2024/BCH_CFP24_solar_wind.csv')

In [13]:
# import plotly.graph_objects as go

# # Plotting the time series using Plotly Graph Objects
# fig = go.Figure()

# fig.add_trace(go.Scatter(x=tsw.index, y=tsw['PeaceRiver_-121.0_56.0'], mode='lines', name='PeaceRiver_-121.0_56.0'))

# fig.update_layout(title="Time Series for PeaceRiver_-121.0_56.0", xaxis_title="Date", yaxis_title="Value")
# fig.show()


In [14]:
# Perform a spatial join to assign the polygon index to the points
joined_gdf = gpd.sjoin(CFP24, cells, how="left", op="intersects")

# The index of the polygon can now be accessed from the resulting joined GeoDataFrame
# Create a new column in the points GeoDataFrame with the index of the polygon
CFP24['ERA5_cell'] = joined_gdf.index_right  # 'in
CFP24.to_file('ROD_2024/BC_CFP24.geojson', driver='GeoJSON')

ValueError: cannot insert ERA5_cell, already exists

In [ ]:
CFP24.explore('ERA5_cell')

# Set Required Args to Activate Modules

In [ ]:
# Iterate over provinces for both solar and wind resources
resource_types = ['wind','solar'] # , 
provinces=['BC']  #'AB','SK','ON','NS'
for province_code in provinces:
    for resource_type in resource_types:
        required_args = {
            "config_file_path": 'config/config.yaml',
            "province_short_code": province_code,
            "resource_type": resource_type
        }
        
        # Create an instance of Resources and execute the module
        res_module = RES.RESources_builder(**required_args)

In [ ]:
cells=datahandler.from_store('cells')

[Assumptions](https://www.bchydro.com/content/dam/BCHydro/customer-portal/documents/corporate/regulatory-planning-documents/integrated-resource-plans/current-plan/rou-characterization-wind-report-20150519-hatch.pdf)
The following assumptions were considered as common to all regions: 
• 3MW turbine model  
• On-shore projects (land-based)
• Project  lifetime of 20 years  
• Crown Land only  
• Costs in 2024$ (CAD) assuming delivery in 2025 
• Project development timelines can vary and the costs for pre construction of the  template project will be based on 5 years of pre development work until reception of  electricity purchase agreement (EPA) and beginning of construction 
• When considering the developer’s internal costs, Hatch has assumed a development 
model where all pre-development costs are absorbed by the same stakeholder that is 
involved in the later stages of the project, including development, procurement, 
construction, BOP, O&M, etc.   This model was chosen over a model where, for 
example, a project has undergone some level of pre-development and then the 
project rights and assets are sold to a new stakeholder prior to construction or 
operation.  Such transaction costs cannot be easily or simply quantified and so Hatch 
considered a simplified model for the purposes of this mandate. Costs associated 
with procuring assets from another developer are not evaluated.   

• The developer is assumed to be experienced, working in a competitive market and to 
be medium or average in size (i.e. not a public utility and not a private, small scale 
independent developer with limited experience)